In [2]:
import pandas as pd
import numpy as np
import os

In [3]:
# Importing the data from IMDB website
title_basics = pd.read_csv('https://datasets.imdbws.com/title.basics.tsv.gz', low_memory=False, sep='\t')
title_basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


In [4]:
# Importing title.akas dataset from IMDB
akas = pd.read_csv('https://datasets.imdbws.com/title.akas.tsv.gz', low_memory=False, sep='\t')
akas.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,1,Карменсіта,UA,\N,imdbDisplay,\N,0
1,tt0000001,2,Carmencita,DE,\N,\N,literal title,0
2,tt0000001,3,Carmencita - spanyol tánc,HU,\N,imdbDisplay,\N,0
3,tt0000001,4,Καρμενσίτα,GR,\N,imdbDisplay,\N,0
4,tt0000001,5,Карменсита,RU,\N,imdbDisplay,\N,0


In [5]:
# Importing title_ratings dataset as pd dataframe
title_ratings = pd.read_csv('https://datasets.imdbws.com/title.ratings.tsv.gz', low_memory=False, sep='\t')
# Displaying the first 5 rows of the dataframe
title_ratings.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,1965
1,tt0000002,5.8,262
2,tt0000003,6.5,1805
3,tt0000004,5.6,178
4,tt0000005,6.2,2603


# Title_basic cleaning process

1. Replace "\N" with np.nan

In [6]:
# Replacing all string values that contains \N with numpy NaN
title_basics = title_basics.replace({'\\N': np.nan})
# Checking to see if "\N" is still in the data
title_basics.isin(['\\N']).any()


tconst            False
titleType         False
primaryTitle      False
originalTitle     False
isAdult           False
startYear         False
endYear           False
runtimeMinutes    False
genres            False
dtype: bool

2. Eliminate movies that are null for runtimeMinutes

3. Eliminate movies that are null for genre

In [7]:
# Dropping all row that contains NaN values in runtimeMinutes and genres
title_basics.dropna(subset=['runtimeMinutes', 'genres'], inplace=True)
# Checking to see if nan values are still in the data
title_basics.isna().sum()

tconst                  0
titleType               0
primaryTitle            1
originalTitle           1
isAdult                 0
startYear          164511
endYear           2744935
runtimeMinutes          0
genres                  0
dtype: int64

4. Keep only titleType==Movie

In [8]:
# Filtering out all titleType that are not movies
keepers = title_basics['titleType'] == 'movie'
# Applying the filter to the dataframe
basics_f = title_basics[keepers]
# Checking to see if the dataframe is filtered correctly
basics_f.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
8,tt0000009,movie,Miss Jerry,Miss Jerry,0,1894,NaN,45,Romance
144,tt0000147,movie,The Corbett-Fitzsimmons Fight,The Corbett-Fitzsimmons Fight,0,1897,NaN,100,"Documentary,News,Sport"
570,tt0000574,movie,The Story of the Kelly Gang,The Story of the Kelly Gang,0,1906,NaN,70,"Action,Adventure,Biography"
587,tt0000591,movie,The Prodigal Son,L'enfant prodigue,0,1907,NaN,90,Drama
672,tt0000679,movie,The Fairylogue and Radio-Plays,The Fairylogue and Radio-Plays,0,1908,NaN,120,"Adventure,Fantasy"


5. Keep startYear 2000-2022

In [9]:
# Creating filter to only keep movies that were released between 2000 and 2022
keepers = (basics_f['startYear'] >= '2000') & (basics_f['startYear'] <= '2022')
# Applying filter to dataframe
basics_f = basics_f[keepers]
# Checking to see if filter worked
basics_f.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
13082,tt0013274,movie,Istoriya grazhdanskoy voyny,Istoriya grazhdanskoy voyny,0,2021,NaN,94,Documentary
34803,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,NaN,118,"Comedy,Fantasy,Romance"
61116,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020,NaN,70,Drama
67669,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,NaN,122,Drama
76059,tt0077684,movie,Histórias de Combóios em Portugal,Histórias de Combóios em Portugal,0,2022,NaN,46,Documentary


6. Eliminate Movies that include "Documentary" in genre

In [10]:
# Using str.contains to remove all movies that are documentaries
documentary = basics_f['genres'].str.contains('Documentary', case=False)
# Using ~ to remove all rows that are True
basics_f = basics_f[~documentary]
# Checking if the rows were removed
basics_f.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
34803,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,NaN,118,"Comedy,Fantasy,Romance"
61116,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020,NaN,70,Drama
67669,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,NaN,122,Drama
77964,tt0079644,movie,November 1828,November 1828,0,2001,NaN,140,"Drama,War"
86801,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005,NaN,100,"Comedy,Horror,Sci-Fi"


# AKAs cleaning and processing

1. Keep only US movies

In [11]:
# Creating filter to only keep US region
df_filer = akas['region'] == 'US'
# Applying filter to dataframe
akas_f = akas[df_filer]
# Checking first 5 rows
akas_f.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
5,tt0000001,6,Carmencita,US,\N,imdbDisplay,\N,0
14,tt0000002,7,The Clown and His Dogs,US,\N,\N,literal English title,0
33,tt0000005,10,Blacksmith Scene,US,\N,imdbDisplay,\N,0
36,tt0000005,1,Blacksmithing Scene,US,\N,alternative,\N,0
41,tt0000005,6,Blacksmith Scene #1,US,\N,alternative,\N,0


2. Replace "\N" with np.nan

In [12]:
# Replacing all string values that contains \N with numpy NaN
akas_f = akas_f.replace({'\\N': np.nan})

In [13]:
# Checking to see "\N"" values were replaced with NaN
akas_f.isin(['\\N']).any()

titleId            False
ordering           False
title              False
region             False
language           False
types              False
attributes         False
isOriginalTitle    False
dtype: bool

# Title_ratings cleaning and processing

1. Replace "\N" with np.nan (if any)

In [14]:
# First lets check to see if "\N" are in the data
title_ratings.isin(['\\N']).any()

tconst           False
averageRating    False
numVotes         False
dtype: bool

Since there are no instances of "\N" I will not need to finish the step above.

2. Keep only US movies

In [15]:
# Filtering ratings to only include movies that are in the akas_f dataframe
filter = title_ratings['tconst'].isin(akas_f['titleId'])
ratings_f = title_ratings[filter]
ratings_f.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,1965
1,tt0000002,5.8,262
4,tt0000005,6.2,2603
5,tt0000006,5.1,178
6,tt0000007,5.4,816


Now I will perform the last step of title_basics.

### Title_basics 

7. Keep only US movies

In [16]:
# Filtering basics_f to only include movies that are in the akas_f
filter = basics_f['tconst'].isin(akas_f['titleId'])
# Applying the filter to basics_f
basics_f = basics_f[filter]
# Checking to see if the filter worked
basics_f.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
34803,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,NaN,118,"Comedy,Fantasy,Romance"
61116,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020,NaN,70,Drama
67669,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,NaN,122,Drama
86801,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005,NaN,100,"Comedy,Horror,Sci-Fi"
93938,tt0096056,movie,Crime and Punishment,Crime and Punishment,0,2002,NaN,126,Drama


After making sure that all dataframes are formatted correctly, now I will create a Data folder to store all dataframes created.

In [17]:
# Using makedirs to create a folder called Data
os.makedirs('Data/', exist_ok=True)
# Using listdir to check if the folder was created
os.listdir('Data/')

['akas_title.csv.gz',
 'basics_title.csv.gz',
 'final_tmdb_data_2000.csv.gz',
 'final_tmdb_data_2001.csv.gz',
 'ratings_title.csv.gz',
 'tmdb_api_results_2000.json',
 'tmdb_api_results_2001.json']

Now I will create a loop that loops through the dataframes created and converts them to a compress csv file and names the file.

In [45]:
# Looping through the dataframes and saving them as a compressed csv file
dfs = [basics_f, akas_f, ratings_f]
names = ['basics', 'akas', 'ratings']

for df, name in zip(dfs, names):
    df.to_csv(f'Data/{name}_title.csv.gz', index=False, compression='gzip')

The last step to do is make sure that the data saved correctly.

In [46]:
# Checking if the files were saved correctly to the Data folder
basics = pd.read_csv('Data/basics_title.csv.gz', compression='gzip')
# Printing the first 5 rows of the dataframe
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,NaN,118,"Comedy,Fantasy,Romance"
1,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020,NaN,70,Drama
2,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,NaN,122,Drama
3,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005,NaN,100,"Comedy,Horror,Sci-Fi"
4,tt0096056,movie,Crime and Punishment,Crime and Punishment,0,2002,NaN,126,Drama


In [47]:
# Checking if the files were saved correctly to the Data folder
akas = pd.read_csv('Data/akas_title.csv.gz', compression='gzip')
# Printing the first 5 rows of the dataframe
akas.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,6,Carmencita,US,NaN,imdbDisplay,NaN,0.0
1,tt0000002,7,The Clown and His Dogs,US,NaN,NaN,literal English title,0.0
2,tt0000005,10,Blacksmith Scene,US,NaN,imdbDisplay,NaN,0.0
3,tt0000005,1,Blacksmithing Scene,US,NaN,alternative,NaN,0.0
4,tt0000005,6,Blacksmith Scene #1,US,NaN,alternative,NaN,0.0


In [48]:
# Checking if the files were saved correctly to the Data folder
ratings = pd.read_csv('Data/ratings_title.csv.gz', compression='gzip')
# Printing the first 5 rows of the dataframe
ratings.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,1965
1,tt0000002,5.8,262
2,tt0000005,6.2,2603
3,tt0000006,5.1,178
4,tt0000007,5.4,816
